# Few Short Prompting

퓨샷 프롬피팅은 기본적으로 모델에 원하는 작업의 몇 가지 예시를 제시.

(예시를 하나도 제공하지 않을 경우 제로 샷, 하나의 예시만 제공할 경우 원 샷)

## FewShortPromptTemplate

In [5]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo")

examples = [
    {
        "question": "다음 덧셈을 계산하시오. 식: 2+5+3+2=",
        "answer": """
          하나씩 계산해 봅시다.
          2+5는 7입니다.
          7+3은 10입니다.
          10+2는 12입니다.
          최종 결과는: 12
        """
    },
    {
        "question": "다음 뺄셈을 계산하시오. 식: 30-5-3-2=",
        "answer": """
          하나씩 계산해 봅시다.
          30-5는 25입니다.
          25-3은 22입니다.
          22-2는 20입니다.
          최종 결과는: 20
        """
    },
    {
        "question": "다음 곱셈을 계산하시오. 식: 2*5*2*1=",
        "answer": """
          하나씩 계산해 봅시다.
          2*5는 10입니다.
          10*2은 20입니다.
          20*1는 20입니다.
          최종 결과는: 20
        """
    }
]

example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question}",
    input_variables=["question"]
)

question = "다음 나눗셈을 계산하시오. 식: 10/5/1="

final_prompt = prompt.format(question=question)

print(llm.predict(final_prompt))


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


          하나씩 계산해 봅시다.
          10/5는 2입니다.
          2/1은 2입니다.
          최종 결과는: 2


## SemanticSimilarityExampleSelector

의미적 유사성을 가지는 예시를 선택.


In [9]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

examples = [
    {
        "question": "다음 덧셈을 계산하시오. 식: 2+5+3+2=",
        "answer": """
          하나씩 계산해 봅시다.
          2+5는 7입니다.
          7+3은 10입니다.
          10+2는 12입니다.
          최종 결과는: 12
        """
    },
    {
        "question": "다음 뺄셈을 계산하시오. 식: 30-5-3-2=",
        "answer": """
          하나씩 계산해 봅시다.
          30-5는 25입니다.
          25-3은 22입니다.
          22-2는 20입니다.
          최종 결과는: 20
        """
    },
    {
        "question": "다음 곱셈을 계산하시오. 식: 2*5*2*1=",
        "answer": """
          하나씩 계산해 봅시다.
          2*5는 10입니다.
          10*2은 20입니다.
          20*1는 20입니다.
          최종 결과는: 20
        """
    },
    {
        "question": "다음 뺄셈을 계산하시오. 식: 20-5-2-1=",
        "answer": """
          하나씩 계산해 봅시다.
          20-5는 15입니다.
          15-2은 13입니다.
          13-1는 12입니다.
          최종 결과는: 12
        """
    }
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,              # 선택 가능한 예시
    OpenAIEmbeddings(),    # 의미적 유사성을 측정하기 위한 임베딩을 생성하는 임베딩 클래스
    Chroma,                # 임베딩을 저장하고 유사성 검색을 수행하는데 사용되는 벡터 스토어
    k=2                    # 선택할 예시의 수
)

question = "다음 나눗셈을 계산하시오. 식: 10-5-1="
selected_examples = example_selector.select_examples({"question": question})
print(f"가장 유사한 예시: {question}\n")

for example in selected_examples:
  print(f"question: {example['question']}")
  print(f"answer: {example['answer']}")

가장 유사한 예시: 다음 나눗셈을 계산하시오. 식: 10-5-1=

question: 다음 뺄셈을 계산하시오. 식: 20-5-2-1=
answer: 
          하나씩 계산해 봅시다.
          20-5는 15입니다.
          15-2은 13입니다.
          13-1는 12입니다.
          최종 결과는: 12
        
question: 다음 뺄셈을 계산하시오. 식: 30-5-3-2=
answer: 
          하나씩 계산해 봅시다.
          30-5는 25입니다.
          25-3은 22입니다.
          22-2는 20입니다.
          최종 결과는: 20
        
